<a href="https://colab.research.google.com/github/ShehabSN/Ocean-Waves/blob/master/550_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz

In [ ]:
!ls

drive				spark-3.0.1-bin-hadoop2.7
ngrok				spark-3.0.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip	spark-3.2.2-bin-hadoop3.2.tgz
ngrok-stable-linux-amd64.zip.1	spark-3.3.1-bin-hadoop2.tgz
sample_data			spark-3.3.1-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.2.2-bin-hadoop3.2.tgz


gzip: stdin: not in gzip format
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyspark[pandas_on_spark] plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-11-28 01:10:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  19.6MB/s    in 0.7s    

2022-11-28 01:10:49 (19.6 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://8cf6-34-125-240-13.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":

In [ ]:
!ls drive/MyDrive/SENG550

UScomments.csv	USvideos.csv


## EDA of USComments.csv

In [ ]:
df_comments = spark.read.csv("drive/MyDrive/SENG550/UScomments.csv", header=True, inferSchema=True)

In [ ]:
df_comments.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- replies: string (nullable = true)



In [ ]:
df_comments.show(5)

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|XpVt6Z1Gjjo|Logan Paul it's y...|    4|      0|
|XpVt6Z1Gjjo|I've been followi...|    3|      0|
|XpVt6Z1Gjjo|Say hi to Kong an...|    3|      0|
|XpVt6Z1Gjjo| MY FAN . attendance|    3|      0|
|XpVt6Z1Gjjo|         trending 😉|    3|      0|
+-----------+--------------------+-----+-------+
only showing top 5 rows



In [ ]:
df_comments.count()

691722

### Removing comments that are empty

In [ ]:
from pyspark.sql.functions import trim
df_comments = df_comments.withColumn("comment_text", trim(df_comments.comment_text))
df_comments.filter(df_comments.comment_text != "").describe().show()

+-------+-----------+--------------------+--------------------+-------------------+
|summary|   video_id|        comment_text|               likes|            replies|
+-------+-----------+--------------------+--------------------+-------------------+
|  count|     691678|              691678|              691351|             691319|
|   mean|       null|1.316029143897996...|  5.8218546351412925|0.42557934326630986|
| stddev|       null|2.055703238922607...|  243.21092767690746|  9.923868799122848|
|    min|--JinobXWPk|                   !|  but you want to...|           Breakups|
|    max|zyPIdeF4NFI|                  🧢|todas as musicas ...|            replies|
+-------+-----------+--------------------+--------------------+-------------------+



## EDA of USVideos.csv

In [ ]:
df_videos = spark.read.csv("drive/MyDrive/SENG550/USvideos.csv", header=True, inferSchema=True)

In [ ]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_total: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- date: string (nullable = true)



In [ ]:
df_videos.show(5)

+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|   video_id|               title|   channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|Logan Paul Vlogs|         24|logan paul vlog|l...|4394029|320053|    5931|        46245|https://i.ytimg.c...|13.09|
|K4wEI5zhHB0|iPhone X — Introd...|           Apple|         28|Apple|iPhone 10|i...|7860119|185853|   26679|            0|https://i.ytimg.c...|13.09|
|cLdxuaxaQwc|         My Response|       PewDiePie|         22|              [none]|5845909|576597|   39774|       170708|https://i.ytimg.c...|13.09|
|WYYvHb03Eog|Apple iPhone X fi...|       The Verge|         28|apple iphone x ha...|2642103| 24975| 

In [ ]:
df_videos.count()

7998

In [ ]:
df_videos.select("views", "likes", "dislikes", "comment_total").describe().show()

+-------+------------------+-----------------+------------------+------------------+
|summary|             views|            likes|          dislikes|     comment_total|
+-------+------------------+-----------------+------------------+------------------+
|  count|              7998|             7998|              7998|              7998|
|   mean| 939101.5788947237|34508.99662415604|1780.8427106776694|4608.3069517379345|
| stddev|2147690.9737737444|91354.65417594886|10818.085919717001|20245.638963905905|
|    min|                 0|                0|                 0|                 0|
|    max|          41500672|          2010366|            318404|            736179|
+-------+------------------+-----------------+------------------+------------------+



### Drop the videos with 0 views or comments

In [ ]:
df_videos_filtered = df_videos.filter(df_videos.views != 0) 
df_videos_filtered = df_videos_filtered.filter(df_videos.comment_total != 0)
df_videos_filtered.select("views", "likes", "dislikes", "comment_total").describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|             views|             likes|          dislikes|     comment_total|
+-------+------------------+------------------+------------------+------------------+
|  count|              7744|              7744|              7744|              7744|
|   mean|  945122.717071281|35332.907412190085|1769.5038739669421| 4759.431559917355|
| stddev|2139395.3499542032| 92383.74832273183|10883.821532341486|20557.540603924426|
|    min|               933|                 0|                 0|                 1|
|    max|          41500672|           2010366|            318404|            736179|
+-------+------------------+------------------+------------------+------------------+



In [ ]:
pd_df_comments = df_comments.pandas_api()
pd_df_videos = df_videos_filtered.pandas_api()

In [ ]:
pd_df_videos.plot.scatter("likes", "dislikes")

In [ ]:
pd_df_videos.plot.scatter("likes", "comment_total")

In [ ]:
pd_df_videos.plot.scatter("dislikes", "comment_total")

### EDA Analysis  
The two above plots show the likes and dislikes versus the amount of comments total. After viewing both plots it is clear to see that the amount of dislikes contributes more towards the amount of comments than the amount of likes. An inference from this is that a video with more dislikes is more likely to have more comments than a video with more likes.